In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Kobe Bryant  shot selection

Cсылка на соревнование: https://www.kaggle.com/c/kobe-bryant-shot-selection

Goal: Fun and education

Using 20 years of data on Kobe's swishes and misses, can you predict which shots will find the bottom of the net? This competition is well suited for practicing classification basics, feature engineering, and time series analysis. Practice got Kobe an eight-figure contract and 5 championship rings. What will it get you?

This data contains the location and circumstances of every field goal attempted by Kobe Bryant took during his 20-year career. Your task is to predict whether the basket went in (shot_made_flag).

We have removed 5000 of the shot_made_flags (represented as missing values in the csv file). These are the test set shots for which you must submit a prediction. You are provided a sample submission file with the correct shot_ids needed for a valid prediction.

In [2]:
data = pd.read_csv('Kobe.csv')

In [3]:
data.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30697 entries, 0 to 30696
Data columns (total 25 columns):
action_type           30697 non-null object
combined_shot_type    30697 non-null object
game_event_id         30697 non-null int64
game_id               30697 non-null int64
lat                   30697 non-null float64
loc_x                 30697 non-null int64
loc_y                 30697 non-null int64
lon                   30697 non-null float64
minutes_remaining     30697 non-null int64
period                30697 non-null int64
playoffs              30697 non-null int64
season                30697 non-null object
seconds_remaining     30697 non-null int64
shot_distance         30697 non-null int64
shot_made_flag        25697 non-null float64
shot_type             30697 non-null object
shot_zone_area        30697 non-null object
shot_zone_basic       30697 non-null object
shot_zone_range       30697 non-null object
team_id               30697 non-null int64
team_name         

пропущенные значения есть только в таргете, которые мы должны предсказать

In [5]:
data['combined_shot_type'].value_counts()

Jump Shot    23485
Layup         5448
Dunk          1286
Tip Shot       184
Hook Shot      153
Bank Shot      141
Name: combined_shot_type, dtype: int64

In [6]:
data_for_an=data.dropna()

In [7]:
data_for_an.shape

(25697, 25)

 Описание данных
 Группа 1.Координаты
 -широта/долгота
 -loc_x/loc_y
 -shot_distance
 -shot_type
 Координаты географические и декартовы по сути содержат одну и ту же информацию-место точки удара на карте.
 Можно и нужно (для линейной регрессии) выбрать что-то одно, что по каким-то причинам будет работать лучше.
 shot distance и shot type то же одно и тоже- двух-или трехочковый бросок зависит от дальности.
 Информация об этом тоже содержится в первых двух переменных. Только еще в первых двух переменных содержится информация об угле.
 Группа 2.Переменные для группировки координат
 -shot_zone_area
 -shot_zone_basic
 -shot_zone_range
 В чем-то содержат пересекающуюся информацию, в чем-то разную. Характеризуют область на поле. Можно сгруппировать по ним координаты.
 Группа 3 Сезон
 - из даты игры вытянуть месяц - чтобы понять, если зависимость от части сезона (начало или конец)
 - season- года- посмотреть количество бросков на количество игр по сезонам.
 Группа 4 Время в игре
 Период+минуты (начало периода/середина/конец)
 Группа 3 (месяц) и 4 обработать как качественные переменные
 Группа 5. Тип броска
 action_type и combined_shot_type Лучше использовать второе (качественная переменная)

Проверим на выбросы и разделим на тест и трейн

In [8]:
data_for_an.describe(percentiles=(0.01, 0.99))

,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,seconds_remaining,shot_distance,shot_made_flag,team_id,shot_id
count,25697.000000,2.569700e+04,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,2.569700e+04,25697.000000
mean,249.348679,2.474109e+07,33.953043,7.148422,91.257345,-118.262652,4.886796,2.520800,0.146243,28.311554,13.457096,0.446161,1.610613e+09,15328.166946
std,149.778520,7.738108e+06,0.088152,110.073147,88.152106,0.110073,3.452475,1.151626,0.353356,17.523392,9.388725,0.497103,0.000000e+00,8860.462397
min,2.000000,2.000001e+07,33.253300,-250.000000,-44.000000,-118.519800,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.610613e+09,2.000000
1%,5.000000,2.000021e+07,33.773300,-225.000000,-18.000000,-118.494800,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.610613e+09,310.960000
50%,253.000000,2.090034e+07,33.970300,0.000000,74.000000,-118.269800,5.000000,3.000000,0.000000,28.000000,15.000000,0.000000,1.610613e+09,15336.000000
99%,545.000000,4.990003e+07,34.062300,228.000000,271.000000,-118.041800,11.000000,5.000000,1.000000,59.000000,28.000000,1.000000,1.610613e+09,30378.080000
max,653.000000,4.990009e+07,34.088300,248.000000,791.000000,-118.021800,11.000000,7.000000,1.000000,59.000000,79.000000,1.000000,1.610613e+09,30697.000000


Из переменных для анализа loc_y имеет выброс и shot_distance

In [9]:
data_for_an=data_for_an[data_for_an['loc_y']<272]

In [10]:
data_for_an.describe(percentiles=(0.01, 0.99))

,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,seconds_remaining,shot_distance,shot_made_flag,team_id,shot_id
count,25442.000000,2.544200e+04,25442.000000,25442.000000,25442.000000,25442.000000,25442.000000,25442.000000,25442.000000,25442.000000,25442.000000,25442.000000,2.544200e+04,25442.000000
mean,248.897257,2.473374e+07,33.955879,7.236459,88.421468,-118.262564,4.921193,2.520439,0.146136,28.484985,13.208474,0.449139,1.610613e+09,15321.976889
std,149.776246,7.731662e+06,0.082943,110.138580,82.943264,0.110139,3.441681,1.151813,0.353250,17.447462,9.009959,0.497416,0.000000e+00,8855.274562
min,2.000000,2.000001e+07,33.773300,-250.000000,-44.000000,-118.519800,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.610613e+09,2.000000
1%,5.000000,2.000021e+07,33.785300,-225.000000,-18.000000,-118.494800,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.610613e+09,311.410000
50%,253.000000,2.090033e+07,33.973300,0.000000,71.000000,-118.269800,5.000000,3.000000,0.000000,28.000000,15.000000,0.000000,1.610613e+09,15318.500000
99%,544.000000,4.990003e+07,34.062300,228.000000,259.000000,-118.041800,11.000000,5.000000,1.000000,59.000000,27.000000,1.000000,1.610613e+09,30376.590000
max,653.000000,4.990009e+07,34.088300,248.000000,271.000000,-118.021800,11.000000,7.000000,1.000000,59.000000,31.000000,1.000000,1.610613e+09,30697.000000


In [11]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data_for_an, test_size=0.2)
len(data_train)

20353

In [12]:
groups_coordinates=data_train.groupby(['shot_zone_area','shot_zone_basic','shot_zone_range']).mean()[['loc_x','loc_y']]

In [13]:
groups_coordinates

loc_x  \
shot_zone_area        shot_zone_basic       shot_zone_range               
Center(C)             Above the Break 3     24+ ft.            1.134052   
                      In The Paint (Non-RA) 8-16 ft.           4.213701   
                                            Less Than 8 ft.    3.490285   
                      Mid-Range             16-24 ft.          1.479755   
                                            8-16 ft.          -2.643172   
                      Restricted Area       Less Than 8 ft.    0.407031   
Left Side Center(LC)  Above the Break 3     24+ ft.         -160.915243   
                      Mid-Range             16-24 ft.       -114.416256   
Left Side(L)          In The Paint (Non-RA) 8-16 ft.         -66.016194   
                      Left Corner 3         24+ ft.         -232.512953   
                      Mid-Range             16-24 ft.       -173.907583   
                                            8-16 ft.        -118.175224   
Right Side Center(RC) Above the Break 3     24+ ft.          155.308246   
                      Mid-Range             16-24 ft.        113.704208   
Right Side(R)         In The Paint (Non-RA) 8-16 ft.          68.551205   
                      Mid-Range             16-24 ft.        173.520428   
                                            8-16 ft.         119.532180   
                      Right Corner 3        24+ ft.          231.238267   

                                                                  loc_y  
shot_zone_area        shot_zone_basic       shot_zone_range              
Center(C)             Above the Break 3     24+ ft.          252.716487  
                      In The Paint (Non-RA) 8-16 ft.         107.196319  
                                            Less Than 8 ft.   43.733161  
                      Mid-Range             16-24 ft.        190.764417  
                                            8-16 ft.         148.158590  
                      Restricted Area       Less Than 8 ft.    3.138924  
Left Side Center(LC)  Above the Break 3     24+ ft.          196.611757  
                      Mid-Range             16-24 ft.        152.054187  
Left Side(L)          In The Paint (Non-RA) 8-16 ft.          75.360324  
                      Left Corner 3         24+ ft.           19.880829  
                      Mid-Range             16-24 ft.         49.347156  
                                            8-16 ft.          41.531377  
Right Side Center(RC) Above the Break 3     24+ ft.          198.169503  
                      Mid-Range             16-24 ft.        153.075495  
Right Side(R)         In The Paint (Non-RA) 8-16 ft.          76.021084  
                      Mid-Range             16-24 ft.         41.909533  
                                            8-16 ft.          35.703114  
                      Right Corner 3        24+ ft.           18.888087

In [14]:
data_train = pd.merge(data_train, groups_coordinates,  how='left', left_on=['shot_zone_area', 'shot_zone_basic', 'shot_zone_range'], right_on = ['shot_zone_area', 'shot_zone_basic', 'shot_zone_range'])

In [15]:
data_test = pd.merge(data_test, groups_coordinates,  how='left', left_on=['shot_zone_area', 'shot_zone_basic', 'shot_zone_range'], right_on = ['shot_zone_area', 'shot_zone_basic', 'shot_zone_range'])

In [16]:
from datetime import datetime
data_1=[]
for i in data_train['game_date'].values:
    a=datetime.strptime(i, '%Y-%m-%d')
    a=a.month
    data_1.append(a)
data_train['month']=data_1
data_2=[]
for i in data_test['game_date'].values:
    a=datetime.strptime(i, '%Y-%m-%d')
    a=a.month
    data_2.append(a)
data_test['month']=data_2

In [17]:
data_season=data_train.groupby(['season','game_id']).count()

In [18]:
data_season=data_season.groupby('season').count()[['action_type','lat']]

In [19]:
data_season_1=data_train.groupby(['season']).count()[['action_type','lat']]

In [20]:
data_season=data_season.merge(data_season_1, how='left', left_on='season', right_on='season')
data_season['shot_per_game']=data_season['lat_y']/data_season['lat_x']
data_season['shots']=data_season['lat_y']
data_season=data_season[['shot_per_game','shots']]

In [21]:
data_train = pd.merge(data_train, data_season,  how='left', left_on=['season'], right_on = ['season'])
data_test = pd.merge(data_test, data_season,  how='left', left_on=['season'], right_on = ['season'])

In [22]:
def minutes_category(db):
    minutes_cat=[]
    for i in db['minutes_remaining'].values:
        if i<=4:
            minutes_cat.append('finish')
        else:
            if i<=8:
                minutes_cat.append('middle')  
            else:
                minutes_cat.append('start')
    print (len(minutes_cat))
    db['minutes_cat']=minutes_cat

In [23]:
minutes_category(data_train)
minutes_category(data_test)

20353
5089


In [24]:
period_str=[]
for i in data_train['period']:
    i=str(i)
    period_str.append(i)

In [25]:
data_train['period_str']=period_str
data_train['period_min']=data_train['period_str']+data_train['minutes_cat']

In [26]:
period_str=[]
for i in data_test['period']:
    i=str(i)
    period_str.append(i)
data_test['period_str']=period_str
data_test['period_min']=data_test['period_str']+data_test['minutes_cat']

In [27]:
data_train['locxLocy']=data_train['loc_x_y']*data_train['loc_y_y']
data_test['locxLocy']=data_test['loc_x_y']*data_test['loc_y_y']

In [28]:
data_train_for_an=data_train[['combined_shot_type','loc_x_y','loc_y_y','locxLocy','month','shot_per_game', 'period_min','shot_made_flag']]

In [29]:
data_test_for_an=data_test[['combined_shot_type','loc_x_y','loc_y_y','locxLocy','month','shot_per_game', 'period_min','shot_made_flag']]

In [30]:
def get_woe_v1(db, db1, col, target_col):
    all_good = len(db[db[target_col] == 1][col])
    all_bad = len(db[db[target_col] == 0][col])
    odds_series = (
        db[db[target_col] == 1][col].value_counts()
        /
        db[db[target_col] == 0][col].value_counts()
    )
    odds_series = odds_series / all_good * all_bad
    category_woe_dict = np.log(odds_series).to_dict()
    db[col + '_woe'] = db[col].apply(category_woe_dict.get)
    db1[col + '_woe'] = db1[col].apply(category_woe_dict.get)
    return db, db1

In [31]:
res=['combined_shot_type', 'month', 'period_min']
for i in res:
    get_woe_v1(data_train_for_an, data_test_for_an, i, 'shot_made_flag')


C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [32]:
data_train_for_an=data_train_for_an.drop(res, axis=1)
data_test_for_an=data_test_for_an.drop(res, axis=1)

In [33]:
corr_matrix = data_train_for_an.corr()
corr_matrix

,loc_x_y,loc_y_y,locxLocy,shot_per_game,shot_made_flag,combined_shot_type_woe,month_woe,period_min_woe
loc_x_y,1.000000,-0.016897,0.823246,0.019316,-0.003805,-0.027659,-0.003621,0.000813
loc_y_y,-0.016897,1.000000,0.023960,0.029670,-0.141776,-0.433516,-0.000627,-0.054917
locxLocy,0.823246,0.023960,1.000000,0.014125,0.004698,-0.015378,-0.005406,0.000688
shot_per_game,0.019316,0.029670,0.014125,1.000000,0.011161,-0.038647,0.008120,0.000972
shot_made_flag,-0.003805,-0.141776,0.004698,0.011161,1.000000,0.242649,0.025966,0.047219
combined_shot_type_woe,-0.027659,-0.433516,-0.015378,-0.038647,0.242649,1.000000,0.000933,0.010731
month_woe,-0.003621,-0.000627,-0.005406,0.008120,0.025966,0.000933,1.000000,0.003527
period_min_woe,0.000813,-0.054917,0.000688,0.000972,0.047219,0.010731,0.003527,1.000000


In [34]:
from sklearn.linear_model import Lasso
from sklearn.metrics import roc_auc_score

def lasso_regression(table, predictors, alpha):
    lassoreg = Lasso(alpha=alpha, normalize=True, max_iter=1e5)
    lassoreg.fit(predictors, table['shot_made_flag'])
    y_pred = lassoreg.predict(predictors)
    roc_auc = np.round(roc_auc_score(table['shot_made_flag'], y_pred), 2)
    rss = sum((y_pred-table['shot_made_flag']) ** 2)
    ret = [rss, roc_auc]
    ret.extend([lassoreg.intercept_])
    ret.extend(lassoreg.coef_)
    return ret

In [35]:
predictors=data_train_for_an.drop('shot_made_flag', axis=1)
predictors1=data_test_for_an.drop('shot_made_flag', axis=1)

In [36]:
alpha_lasso = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

# create dataframe to store coefficients
col = ['rss','roc_auc', 'intercept'] + predictors.columns.tolist()
ind = ['alpha_%.2g' % alpha_lasso[i] for i in range(0, 10)]
coef_matrix_lasso = pd.DataFrame(index=ind, columns=col)

#Iterate over the 10 alpha values:
for i in range(10):
    coef_matrix_lasso.iloc[i, ] = lasso_regression(data_train_for_an, predictors, alpha_lasso[i])

In [37]:
coef_matrix_lasso

,rss,roc_auc,intercept,loc_x_y,loc_y_y,locxLocy,shot_per_game,combined_shot_type_woe,month_woe,period_min_woe
alpha_1e-15,4711.66,0.61,0.406309,-8.82862e-05,-0.000279543,8.77971e-07,0.00442389,0.177673,0.242051,0.219558
alpha_1e-10,4711.66,0.61,0.406309,-8.82854e-05,-0.000279543,8.77965e-07,0.00442388,0.177673,0.242051,0.219558
alpha_1e-08,4711.66,0.61,0.406316,-8.82086e-05,-0.000279525,8.77377e-07,0.00442323,0.177672,0.242024,0.219544
alpha_1e-05,4712.32,0.61,0.413444,-1.14363e-05,-0.000261594,2.8867e-07,0.00377315,0.176458,0.214866,0.205609
alpha_0.0001,4727.69,0.61,0.456194,0,-0.000153317,0,0,0.16145,0,0.0779783
alpha_0.001,5031.73,0.5,0.447305,-0,-0,0,0,0,0,0
alpha_0.01,5031.73,0.5,0.447305,-0,-0,0,0,0,0,0
alpha_1,5031.73,0.5,0.447305,-0,-0,0,0,0,0,0
alpha_5,5031.73,0.5,0.447305,-0,-0,0,0,0,0,0
alpha_10,5031.73,0.5,0.447305,-0,-0,0,0,0,0,0


Обнуляются месяц, среднее количество ударов за игру, координата X и совокупная координата

In [38]:
# create dataframe to store coefficients
col = ['rss','roc_auc', 'intercept'] + predictors.columns.tolist()
ind = ['alpha_%.2g' % alpha_lasso[i] for i in range(0, 10)]
coef_matrix_lasso_test = pd.DataFrame(index=ind, columns=col)

#Iterate over the 10 alpha values:
for i in range(10):
    coef_matrix_lasso_test.iloc[i, ] = lasso_regression(data_test_for_an, predictors1, alpha_lasso[i])
coef_matrix_lasso_test

,rss,roc_auc,intercept,loc_x_y,loc_y_y,locxLocy,shot_per_game,combined_shot_type_woe,month_woe,period_min_woe
alpha_1e-15,1179.71,0.62,0.37997,9.28897e-05,-0.000223016,1.50333e-07,0.00650981,0.185912,-0.224714,0.256857
alpha_1e-10,1179.71,0.62,0.37997,9.28897e-05,-0.000223016,1.50332e-07,0.00650981,0.185912,-0.224714,0.256856
alpha_1e-08,1179.71,0.62,0.379973,9.28861e-05,-0.00022301,1.50303e-07,0.0065095,0.185911,-0.2247,0.25685
alpha_1e-05,1179.72,0.62,0.383783,8.93555e-05,-0.00021678,1.20344e-07,0.00619881,0.185019,-0.211242,0.250301
alpha_0.0001,1181.1,0.62,0.418305,4.19914e-05,-0.000162313,0,0.00339737,0.176884,-0.0900796,0.190936
alpha_0.001,1212.99,0.59,0.453995,0,-0,0,0,0.0815617,-0,0
alpha_0.01,1262.61,0.5,0.456475,0,-0,0,0,0,-0,0
alpha_1,1262.61,0.5,0.456475,0,-0,0,0,0,-0,0
alpha_5,1262.61,0.5,0.456475,0,-0,0,0,0,-0,0
alpha_10,1262.61,0.5,0.456475,0,-0,0,0,0,-0,0


In [39]:
from sklearn.linear_model import Ridge

def ridge_regression(data, predictors, alpha):
    ridgereg = Ridge(alpha=alpha, normalize=True)
    ridgereg.fit(predictors, data['shot_made_flag'])
    y_pred = ridgereg.predict(predictors)
    rss = sum((y_pred-data['shot_made_flag']) ** 2)
    roc_auc = np.round(roc_auc_score(data['shot_made_flag'], y_pred), 2)
    ret = [rss, roc_auc]
    ret.extend([ridgereg.intercept_])
    ret.extend(ridgereg.coef_)
    return ret

In [40]:
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]
col = ['rss','roc_auc', 'intercept'] + predictors.columns.tolist()
ind = ['alpha_%.2g' % alpha_ridge[i] for i in range(0, 10)]
coef_matrix_ridge = pd.DataFrame(index=ind, columns=col)

for i in range(10):
    coef_matrix_ridge.iloc[i, ] = ridge_regression(data_train_for_an, predictors, alpha_ridge[i])

In [41]:
coef_matrix_ridge

,rss,roc_auc,intercept,loc_x_y,loc_y_y,locxLocy,shot_per_game,combined_shot_type_woe,month_woe,period_min_woe
alpha_1e-15,4711.66,0.61,0.406309,-8.82862e-05,-0.000279543,8.77971e-07,0.00442389,0.177673,0.242051,0.219558
alpha_1e-10,4711.66,0.61,0.406309,-8.82862e-05,-0.000279543,8.77971e-07,0.00442389,0.177673,0.242051,0.219558
alpha_1e-08,4711.66,0.61,0.406309,-8.82862e-05,-0.000279543,8.77971e-07,0.00442388,0.177673,0.242051,0.219558
alpha_0.0001,4711.66,0.61,0.406322,-8.82378e-05,-0.000279582,8.77571e-07,0.00442327,0.177653,0.242027,0.219535
alpha_0.001,4711.66,0.61,0.406432,-8.78051e-05,-0.000279933,8.73988e-07,0.00441777,0.177475,0.241813,0.219335
alpha_0.01,4711.69,0.61,0.407519,-8.37023e-05,-0.000283338,8.39914e-07,0.00436348,0.175714,0.239688,0.217357
alpha_1,4778.4,0.61,0.446579,-1.31904e-05,-0.000294864,1.74489e-07,0.00176863,0.0883788,0.122116,0.112716
alpha_5,4915.76,0.61,0.451695,-3.0848e-06,-0.000132039,3.7582e-08,0.000467345,0.0309182,0.0409751,0.0394927
alpha_10,4963.45,0.61,0.450326,-1.62624e-06,-7.64503e-05,1.81426e-08,0.000237405,0.0171378,0.0223852,0.0218597
alpha_20,4994.33,0.61,0.449081,-8.4375e-07,-4.14053e-05,8.7843e-09,0.000118859,0.00906729,0.0117363,0.0115532


минимальное количество переменных без снижения качества достигается при alpha_1. Сильнее всего близки к 0 координата x, скомбинированная x-y

In [42]:
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]
col = ['rss','roc_auc', 'intercept'] + predictors.columns.tolist()
ind = ['alpha_%.2g' % alpha_ridge[i] for i in range(0, 10)]
coef_matrix_ridge1 = pd.DataFrame(index=ind, columns=col)

for i in range(10):
    coef_matrix_ridge1.iloc[i, ] = ridge_regression(data_test_for_an, predictors1, alpha_ridge[i])

In [43]:
coef_matrix_ridge1

,rss,roc_auc,intercept,loc_x_y,loc_y_y,locxLocy,shot_per_game,combined_shot_type_woe,month_woe,period_min_woe
alpha_1e-15,1179.71,0.62,0.37997,9.28897e-05,-0.000223016,1.50333e-07,0.00650981,0.185912,-0.224714,0.256857
alpha_1e-10,1179.71,0.62,0.37997,9.28897e-05,-0.000223016,1.50333e-07,0.00650981,0.185912,-0.224714,0.256857
alpha_1e-08,1179.71,0.62,0.37997,9.28897e-05,-0.000223016,1.50333e-07,0.00650981,0.185912,-0.224714,0.256857
alpha_0.0001,1179.71,0.62,0.379987,9.28549e-05,-0.000223074,1.50539e-07,0.006509,0.18589,-0.224691,0.25683
alpha_0.001,1179.71,0.62,0.380139,9.2543e-05,-0.000223589,1.52379e-07,0.00650169,0.185692,-0.224483,0.25659
alpha_0.01,1179.71,0.62,0.38164,8.95958e-05,-0.000228582,1.69499e-07,0.00642946,0.183748,-0.222425,0.254227
alpha_1,1197.41,0.62,0.439397,3.46974e-05,-0.000280555,2.33622e-07,0.00279388,0.0909817,-0.110472,0.131993
alpha_5,1232.94,0.61,0.455632,1.26452e-05,-0.00012859,8.6559e-08,0.000797817,0.0316522,-0.0363798,0.046365
alpha_10,1245.16,0.61,0.45667,7.06679e-06,-7.47036e-05,4.80337e-08,0.000415888,0.017527,-0.0197859,0.0256741
alpha_20,1253.06,0.61,0.456779,3.75365e-06,-4.05293e-05,2.5387e-08,0.000211784,0.00926826,-0.0103464,0.0135718


In [44]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(predictors, data_train_for_an['shot_made_flag'])
y_pred = lin_reg.predict(predictors)
roc_auc = np.round(roc_auc_score(data_train_for_an['shot_made_flag'], y_pred), 2)
rss = sum((y_pred-data_train_for_an['shot_made_flag']) ** 2)
print (rss, roc_auc)

4711.661297090935 0.61


In [45]:
lin_reg.fit(predictors1, data_test_for_an['shot_made_flag'])
y_pred1 = lin_reg.predict(predictors1)
roc_auc = np.round(roc_auc_score(data_test_for_an['shot_made_flag'], y_pred1), 2)
rss = sum((y_pred1-data_test_for_an['shot_made_flag']) ** 2)
print (rss, roc_auc)

1179.7063799619928 0.62


In [46]:
data_train.columns

Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x_x', 'loc_y_x', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'season', 'seconds_remaining', 'shot_distance', 'shot_made_flag',
       'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'team_id', 'team_name', 'game_date', 'matchup', 'opponent', 'shot_id',
       'loc_x_y', 'loc_y_y', 'month', 'shot_per_game', 'shots', 'minutes_cat',
       'period_str', 'period_min', 'locxLocy'],
      dtype='object')

In [47]:

data_train_fac=data_train.drop(['action_type','period', 'game_event_id', 'game_id','season','minutes_remaining','shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
'team_id', 'team_name',  'matchup', 'shot_id','shot_per_game',  'game_date','minutes_cat','period_str'], axis=1)
data_test_fac=data_test.drop(['action_type','period', 'game_event_id', 'game_id','minutes_remaining','season','shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'team_id', 'team_name',  'matchup', 'shot_id',
        'shot_per_game',  'game_date','minutes_cat',
       'period_str'], axis=1)
res=['combined_shot_type', 'month', 'period_min', 'opponent']
for i in res:
    get_woe_v1(data_train_fac, data_test_fac, i, 'shot_made_flag')
predictors2=data_train_fac.drop('shot_made_flag', axis=1)
predictors2=predictors2.drop(res,axis=1)
from sklearn.decomposition import PCA
pca = PCA(n_components=predictors2.shape[1]).fit(predictors2)
for i in range(1, predictors2.shape[1] + 1):
    print("n-components=%s, explained variance %.3f" % (i, sum(pca.explained_variance_ratio_[:i])))

n-components=1, explained variance 0.999
n-components=2, explained variance 1.000
n-components=3, explained variance 1.000
n-components=4, explained variance 1.000
n-components=5, explained variance 1.000
n-components=6, explained variance 1.000
n-components=7, explained variance 1.000
n-components=8, explained variance 1.000
n-components=9, explained variance 1.000
n-components=10, explained variance 1.000
n-components=11, explained variance 1.000
n-components=12, explained variance 1.000
n-components=13, explained variance 1.000
n-components=14, explained variance 1.000
n-components=15, explained variance 1.000


In [48]:
new_factors = PCA(n_components=2).fit_transform(predictors2)
model2 = LinearRegression().fit(new_factors, data_train_fac['shot_made_flag'])
model2.score(new_factors, data_train_fac['shot_made_flag'])

0.0008077053646258214

плохо описывают

следующий этап - проанализировать решения участников на кагле. Больше всего понра решение с площадями.
https://www.kaggle.com/selfishgene/psychology-of-a-professional-athlete
Хоть там и подход несколько иной, и информации о качестве я не нашла, но истина где-то в этом
Качество других решений варьируется от 0.6 до 0.7, те манипуляции с данными не приносят существенных результатов.
Каждый делает по -своему. а качесвто одно и то же.

В том примере исследователь считает площади и процент успешных попыток.
По аналогии, но более тупо- просто посчитаем WOE по сгруппир критеряям
shot_zone_area        30697 non-null object
shot_zone_basic       30697 non-null object
shot_zone_range       30697 non-null object

In [49]:
data_train['combined']=data_train['shot_zone_area']+data_train['shot_zone_basic']+data_train['shot_zone_range']
data_test['combined']=data_test['shot_zone_area']+data_test['shot_zone_basic']+data_test['shot_zone_range']

In [50]:
columns_2=['combined_shot_type', 'month', 'period_min','shot_per_game', 'combined', 'shot_made_flag']
data_tr_an2=data_train[columns_2]
data_test_an2=data_test[columns_2]

In [51]:
columns_woe=['combined_shot_type', 'month', 'period_min', 'combined']
for i in columns_woe:
    get_woe_v1(data_tr_an2, data_test_an2, i, 'shot_made_flag')

C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [52]:
data_tr_an2=data_tr_an2.drop(columns_woe, axis=1)
data_test_an2=data_test_an2.drop(columns_woe, axis=1)

In [53]:
predictors_w=data_tr_an2.drop('shot_made_flag', axis=1)
predictors_w1=data_test_an2.drop('shot_made_flag', axis=1)

In [54]:
lin_reg_w = LinearRegression()
lin_reg_w.fit(predictors_w, data_tr_an2['shot_made_flag'])
y_pred_w = lin_reg_w.predict(predictors_w)
roc_auc = np.round(roc_auc_score(data_tr_an2['shot_made_flag'], y_pred_w), 2)
rss = sum((y_pred_w-data_tr_an2['shot_made_flag']) ** 2)
print (rss, roc_auc)

4701.015438510887 0.62


можно сказать, что не лучше. Можно было б, как в примере- выявить типы стратегий игрокам по сегментам поля и ситуациям.
Возможно, это дало бы результат.

**Задания:**

1. Провести анализ данных. Много хороших примеров анализа можно посмотреть здесь https://www.kaggle.com/c/kobe-bryant-shot-selection/kernels
2. Подготовить фичи для обучения модели - нагенерить признаков, обработать пропущенные значения, проверить на возможные выбросы, обработать категориальные признаки и др.
3. Обучить линейную модель, Lasso, Ridge на тех же признаках - построить сравнительную таблицу коэффициентов, сделать заключения о том, как меняется величина коэффициентов, какие зануляются. Посчитать RSS

**Дополнительно**
4. Сравнить результаты на тестовом наборе данных - сделать train_test_split в самом начале, подготовить переменные, сравнить результаты работы классификаторов (те же 3), метрика ROC AUC
5. Построить PCA на подготовленных признаках, посмотреть, какие компоненты составляют наибольшую часть дисперсии целевой переменной